# GNOD Recommender

### Getting started

#### 1. Import libraries

In [1]:
# import libraries
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
import time
from pandas import json_normalize
from sklearn.cluster import KMeans
import numpy as np

#### 2.  Log into Spotify

In [2]:
# Consists of our spotify credentials to log in
secrets_file = open("secrets.txt","r") #r = opening for reading, needs to be specified

# Read the whole file in one go
string = secrets_file.read()

string.split('\n')

# Translate the set of strings in a dictionary
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

#### 3. Import dataframes

#### 1. YES -- Top100 -- Dataframe to give a recommendation if the song and/or artist are in the list**

In [4]:
top100 = pd.read_csv('top_100.csv')
top100

,artist,title
0,taylor swift,anti-hero
1,bizarrap & shakira,"shakira: bzrp music sessions, vol. 53"
2,sam smith & kim petras,unholy
3,jelly roll,son of a sinner
4,lainey wilson,heart like a truck
...,...,...
95,consumed by fire,first things first
96,tyler childers,way of the triune god (hallelujah version)
97,katy nichole & big daddy weave,god is in this story
98,tiësto & tate mcrae,10:35


In [5]:
top100.values

array([['taylor swift', 'anti-hero'],
       ['bizarrap & shakira', 'shakira: bzrp music sessions, vol. 53'],
       ['sam smith & kim petras', 'unholy'],
       ['jelly roll', 'son of a sinner'],
       ['lainey wilson', 'heart like a truck'],
       ['rihanna',
        'lift me up (from black panther: wakanda forever - music from and inspired by)'],
       ['hardy & lainey wilson', 'wait in the truck'],
       ['nick carter', 'hurts to love you'],
       ['meghan trainor', 'made you look'],
       ['sean paul', 'temperature'],
       ['morgan wallen', 'thought you should know'],
       ['jeff beck & rod stewart', 'people get ready'],
       ['daughtry', 'separate ways (worlds apart) [feat. lzzy hale]'],
       ['david guetta & bebe rexha', "i'm good (blue)"],
       ['onerepublic', "i ain't worried"],
       ['kane brown & katelyn brown', 'thank god'],
       ['zach bryan', "something in the orange (z&e's version)"],
       ['sia', 'unstoppable'],
       ['lady gaga', 'bloody mary'],

##### 2. NO -- Data not in TOP 100, make recommendation from Spotify

In [6]:
df_recommender = pd.read_csv("df_recommender.csv")
#df_recommender.head()

,artist,song,popularity,uri,url,danceability,energy,key,loudness,mode,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,0,clusters
0,Taylor Swift,...Ready For It?,0,spotify:track:7zgqtptZvhf8GEmdsM2vp2,https://open.spotify.com/track/7zgqtptZvhf8GEm...,0.615,0.779,2.0,-6.454,1.0,...,0.453,160.000,audio_features,7zgqtptZvhf8GEmdsM2vp2,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198.0,4.0,NaN,8
1,Thomas Rhett,Life Changes,63,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://open.spotify.com/track/4Vxu50qVrQcycjR...,0.687,0.845,7.0,-4.370,1.0,...,0.809,87.972,audio_features,4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227.0,4.0,NaN,9
2,Bruno Mars,24K Magic,81,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://open.spotify.com/track/6b8Be6ljOzmkOmF...,0.818,0.803,1.0,-4.282,1.0,...,0.632,106.970,audio_features,6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983.0,4.0,NaN,9
3,Ed Sheeran,Galway Girl,80,spotify:track:0afhq8XCExXpqazXczTSve,https://open.spotify.com/track/0afhq8XCExXpqaz...,0.624,0.876,9.0,-3.374,1.0,...,0.781,99.943,audio_features,0afhq8XCExXpqazXczTSve,https://api.spotify.com/v1/tracks/0afhq8XCExXp...,https://api.spotify.com/v1/audio-analysis/0afh...,170827.0,4.0,NaN,9
4,Ed Sheeran,Photograph,86,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://open.spotify.com/track/1HNkqx9Ahdgi1Ix...,0.614,0.379,4.0,-10.480,1.0,...,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987.0,4.0,NaN,0


Only **certain information** for the **recommendation** is needed
- artist
- song
- url
- cluster

In [7]:
df_recomm = df_recommender[['artist', 'song', 'url', 'clusters']]
#df_recomm

,artist,song,url,clusters
0,Taylor Swift,...Ready For It?,https://open.spotify.com/track/7zgqtptZvhf8GEm...,8
1,Thomas Rhett,Life Changes,https://open.spotify.com/track/4Vxu50qVrQcycjR...,9
2,Bruno Mars,24K Magic,https://open.spotify.com/track/6b8Be6ljOzmkOmF...,9
3,Ed Sheeran,Galway Girl,https://open.spotify.com/track/0afhq8XCExXpqaz...,9
4,Ed Sheeran,Photograph,https://open.spotify.com/track/1HNkqx9Ahdgi1Ix...,0
...,...,...,...,...
9939,Elton John,Funeral For A Friend / Love Lies Bleeding,https://open.spotify.com/track/4UFlPCB4THnQ9Tl...,1
9940,Supertramp,Fool's Overture,https://open.spotify.com/track/5pSSEkT0963muzz...,0
9941,Yes,Heart of the Sunrise - 2003 Remaster,https://open.spotify.com/track/7gC6Rbllqf1yXNC...,6
9942,Paul McCartney,Venus And Mars / Rock Show / Jet - Live / Rema...,https://open.spotify.com/track/6Ff77WXC58MkhLE...,3


In [8]:
df_clustering = pd.read_csv("df_clustering.csv")

In [9]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(df_clustering)
X_prep = transformer.transform(df_clustering)
pd.DataFrame(X_prep,columns=df_clustering.columns).head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.283739,0.565427,0.279158,0.877677,-0.592032,-0.403138,-0.195119,-0.080821,1.401571,-0.203087,0.185143
1,0.717151,0.858514,0.748519,-0.159008,-0.476902,-0.403138,-0.908790,1.348875,-1.152776,-0.251831,0.185143
2,1.505719,0.672004,0.768338,0.136997,-0.703725,-0.403138,-0.208118,0.638043,-0.479045,-0.154848,0.185143
3,0.337916,0.996175,0.972839,0.408892,-0.567975,-0.403138,0.922837,1.236427,-0.728245,-0.304451,0.185143
4,0.277720,-1.210854,-0.627583,-0.292946,1.265507,-0.401253,-0.561704,-1.092854,-0.442908,-0.065330,0.185143


In [10]:
kmeans = KMeans(n_clusters=10, random_state=1234)
kmeans.fit(X_prep)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10, random_state=1234)

In [11]:
clusters = kmeans.predict(X_prep)

# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()

0    1343
1    2055
2     569
3     602
4     352
5      99
6     406
7     468
8    1462
9    2588
dtype: int64

### Recommender

In [12]:
# Turn everything into lower cases
top100['title']=top100['title'].str.lower()
top100['artist']=top100['artist'].str.lower()
top100.head(2)

,artist,title
0,taylor swift,anti-hero
1,bizarrap & shakira,"shakira: bzrp music sessions, vol. 53"


#### Let's recommend a song

In [37]:
artist = input("Please enter the artist you are listening to? ", ).lower()
title = input("What song are you listening to from this artist? ", ).lower()

print("\n")
music = [artist,title]

if title in top100.values:
    print("Try this song next!")
    print("\n")
    print(str(top100.sample(n=1)).title())
    
elif title not in top100.values:
    try:
        
        # Step 1 - Replace q with music[1] or title
        #results = sp.search(q='title', limit = 1)
        results = sp.search(q='track:' + (''+ title +'') + ', artist:' + (''+ artist +''), type='track', limit=1)

        # Step 2 - Find the uri of that song
        song_uri = results["tracks"]["items"][0]["uri"]

        # Step 3 - Find the uri on spotify
        audio_features = sp.audio_features(song_uri)

        # Step 4 - Get a dataframe
        audio_features = json_normalize(audio_features)
        audio_features = audio_features.drop(["key","mode", 'id','type','track_href','uri', 'analysis_url'], axis = 1)

        # Step 5 - Scale audio_features
        scale_audio_features = transformer.transform(audio_features)

        # Step 6 - Find the cluster
        cluster = kmeans.predict(scale_audio_features)
        cluster = pd.DataFrame(cluster)
        cluster.columns = ['pred_cluster']

        # Step 7 - Chose a random song from the cluster
        pd.options.display.max_colwidth = 300
        songs = pd.merge(left = cluster,
                         right = df_recomm,
                         how = 'inner',
                         left_on = 'pred_cluster',
                         right_on = "clusters")
        songs.style.set_properties(subset=['url'], **{'width': '500px'})
        song_rec = songs[['artist', 'song', 'url']].sample(n=1)
        artist = song_rec['artist'].to_string(index=False)
        song = song_rec['song'].to_string(index=False)
        link = song_rec['url'].to_string(index=False)
        
        print("Try this song next!")
        print("\n")
        print("We recommend the following song " + song + " by " + artist + "." + "\n" + "Listen to it here: " + link)
        
    except:
        print('\nPlease try again. Maybe you misspelled something')
        time.sleep(5) 
else:
    print("Sorry I cannot recommend you a song at the moment")

Please enter the artist you are listening to? taylor swift
What song are you listening to from this artist? ready


Try this song next!


We recommend the following song Cry Thunder by DragonForce.
Listen to it here: https://open.spotify.com/track/45W8En2qul3ljQKzIbsyT4


In [35]:
cluster
#print("We recommend")

,pred_cluster
0,7


In [ ]:
## TRIALS

In [ ]:
# artist = input("Please enter the artist you are listening to?", ).lower()
# title = input("What song are you listening to from this artist?", ).lower()
# print("\n")
# music = [artist,title]

# if music in top100.values:
#     print("Try this song next!")
#     print("\n")
#     print(str(top100.sample(n=1)).title())
    
# elif music[0] in top100.values:
#     print("Try this song next!")
#     print("\n")
#     print(str(top100.sample(n=1)).title())
    
# elif title not in top100.values:
#     # Step 1 - Replace q with music[1] or title
#     results = sp.search(q=title, limit=50)
#     results

#     # Step 2 - Find the uri of that song
#     song_uri = results["tracks"]["items"][0]["uri"]

#     # Step 3 - Find the uri on spotify
#     audio_features = sp.audio_features(song_uri)

#     # Step 4 - Get a dataframe
#     audio_features = json_normalize(audio_features)
#     audio_features = audio_features.drop(["key","mode", 'id','type','track_href','uri', 'analysis_url'], axis = 1)

#     # Step 5 - Scale audio_features
#     scale_audio_features = transformer.transform(audio_features)

#     # Step 6 - Find the cluster
#     cluster = kmeans.predict(scale_audio_features)
#     cluster = pd.DataFrame(cluster)
#     cluster.columns = ['pred_cluster']

#     # Step 7 - Chose a random song from the cluster
#     songs = pd.merge(left = cluster,
#                      right = df_recomm,
#                      how = 'inner',
#                      left_on = 'pred_cluster',
#                      right_on = "clusters")

#     print("Try this song next!")
#     print("\n")
#     print(str(songs[['artist', 'song', 'url']].sample(n=1)).title())
        
# else:
#     print("Sorry I cannot recommend you a song at the moment")

In [ ]:
# # Step 1 - Replace q with music[1] or title
# results = sp.search(q=title, limit = 3)

# # Step 2 - Find the uri of that song
# song_uri = results["tracks"]["items"][0]["uri"]

# # Step 3 - Find the uri on spotify
# audio_features = sp.audio_features(song_uri)

# # Step 4 - Get a dataframe
# audio_features = json_normalize(audio_features)
# audio_features = audio_features.drop(["key","mode", 'id','type','track_href','uri', 'analysis_url'], axis = 1)
# audio_features

# # Step 5 - Scale audio_features
# scale_audio_features = transformer.transform(audio_features)
# scale_audio_features

# # Step 6 - Find the cluster
# cluster = kmeans.predict(scale_audio_features)
# cluster
# cluster = pd.DataFrame(cluster)
# cluster.columns = ['pred_cluster']
# cluster

# # Step 7 - Chose a random song from the cluster
# songs = pd.merge(left = cluster,
#                 right = df_recomm,
#                 how = 'inner',
#                 left_on = 'pred_cluster',
#                 right_on = "clusters")
# songs
# print(str(songs[['artist', 'song', 'url']].sample(n=1)).title())

# #for i in cluster['pred_cluster']:

# #     if i isin([df_recomm["clusters"]]).any():
# #         print("Try this song next!")
# #         print("\n")
# #         print(str(df_recomm["clusters"].sample(n=1)).title())
        